In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import kagglehub
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pathlib
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetV2B1
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import cv2

kaggle_wound_path = kagglehub.dataset_download('yasinpratomo/wound-dataset')
dataset_path = f"{kaggle_wound_path}/Wound_dataset"
print(dataset_path)

for d in dataset_path:
  print(d)

for root, dirs, files in os.walk(dataset_path):
  # print("Directory: ", root)
  for filename in files:
    full_filepath = os.path.join(root, filename)
    # print(f" = {full_filepath}")

In [ ]:
import os
import shutil
import random
from pathlib import Path

# CONFIGURATION
original_dataset_dir = dataset_path
output_base_dir = "dataset_split"
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15
SEED = 123

random.seed(SEED)

# Make output folders
for split in ["train", "val", "test"]:
    split_path = Path(output_base_dir) / split
    split_path.mkdir(parents=True, exist_ok=True)

# Go through each class
for class_name in os.listdir(original_dataset_dir):
    class_dir = os.path.join(original_dataset_dir, class_name)
    if not os.path.isdir(class_dir):
        continue  # skip non-folder items

    images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
    random.shuffle(images)

    total = len(images)
    train_cutoff = int(train_ratio * total)
    val_cutoff = train_cutoff + int(val_ratio * total)

    train_images = images[:train_cutoff]
    val_images = images[train_cutoff:val_cutoff]
    test_images = images[val_cutoff:]

    def move_images(image_list, split):
        split_class_dir = Path(output_base_dir) / split / class_name
        split_class_dir.mkdir(parents=True, exist_ok=True)
        for img_name in image_list:
            src = os.path.join(class_dir, img_name)
            dst = os.path.join(split_class_dir, img_name)
            shutil.copy2(src, dst)

    move_images(train_images, "train")
    move_images(val_images, "val")
    move_images(test_images, "test")

    print(f"Class '{class_name}': {len(train_images)} train, {len(val_images)} val, {len(test_images)} test")

print("Dataset split complete.")


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## image mode = RGB -> 3 channels

In [ ]:
def preprocess(img):
  if img.shape[2] == 3:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224, 224))
  img = img.astype("float32")

  img = preprocess_input(img)
  img = np.expand_dims(img, axis=0)
  return img

In [ ]:
BATCH_SIZE = 32
IMG_WIDTH = 180
IMG_HEIGHT = 180

In [ ]:
train_set = keras.utils.image_dataset_from_directory(
    "dataset_split/train",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

In [ ]:
validation_set = keras.utils.image_dataset_from_directory(
    "dataset_split/val",
    image_size=(IMG_HEIGHT, IMG_HEIGHT),
    batch_size=BATCH_SIZE
)

In [ ]:
test_set = keras.utils.image_dataset_from_directory(
    "dataset_split/test",
    image_size=(IMG_HEIGHT, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False
)

## Data augmentation as there was alot of overfitting (train doing well, val doing not well)

In [ ]:
# data_augmentation = keras.Sequential([
#     layers.RandomFlip("horizontal_and_vertical"),
#     layers.RandomRotation(0.2),
#     layers.RandomZoom(0.1),
#     layers.RandomContrast(0.2),
# ])

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.1, 0.1),
])


In [ ]:
class_names = train_set.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_set.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_set:
  print(f"IMAGE BATCH SHAPE: {image_batch} <-> LABELS BATCH SHAPE: {labels_batch}")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
%pip install optuna

In [ ]:
import optuna
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

tf.keras.backend.clear_session()

def objective(trial):
  optim_learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2)
  optim_dropout_rate = trial.suggest_float("dropout", 0.2, 0.5)
  optim_weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2)
  optim_patience = trial.suggest_int("patience", 5, 10)

  num_classes = len(class_names)


  # base_model = EfficientNetV2B0(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights='imagenet')
  # base_model.trainable = True
  # for layer in base_model.layers[:-20]:
  #   layer.trainable = False

  # inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
  # x = data_augmentation(inputs)
  # x = preprocess_input(x)
  # x = base_model(x, training=False)
  # x = layers.GlobalAveragePooling2D()(x)
  # x = layers.Dropout(optim_dropout_rate)(x)
  # outputs = layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(optim_weight_decay))(x)
  base_model = EfficientNetV2B1(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, 
                                                                                    IMG_WIDTH, 3))
    
  # Freezing all layers except the last 10
  for layer in base_model.layers[:-10]:  # Fine-tune more layers
    layer.trainable = False
    
    # Build the model
  vision_network = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(optim_dropout_rate),
        Dense(num_classes, activation='softmax')  # Output layer
  ])

  vision_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optim_learning_rate),
                        loss=keras.losses.SparseCategoricalCrossentropy(),
                        metrics=["accuracy"])

  print(vision_network.summary())

  EPOCHS = 50

  lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
  early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # setting unique model checkpoint path for each trial (because if overwriting same path, gives error and best hyperparameters won't save)
  checkpoint_path = f"best_model_trial_{trial.number}.keras"
  checkpoint = keras.callbacks.ModelCheckpoint(
  checkpoint_path, save_best_only=True, monitor="val_loss", mode="min"
  )

  history = vision_network.fit(
      train_set,
      validation_data=validation_set,
      epochs=EPOCHS,
      callbacks=[lr_scheduler, early_stopping, checkpoint],
  )

  return min(history.history["val_loss"])

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

In [ ]:
best_trial_number = study.best_trial.number
best_model_path = f"best_model_trial_{best_trial_number}.keras"

best_model = tf.keras.models.load_model(best_model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved as model.tflite (from trial #{best_trial_number})")


## reusing best params to compile model and retraining ->  then plotting accuracy

In [ ]:
best_parameters = study.best_trial.params

# rebuild the model using the best parameters found
base_model = EfficientNetV2B0(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights='imagenet')
base_model.trainable = False

inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(best_parameters["dropout"])(x)
outputs = layers.Dense(
    len(class_names), 
    activation='softmax', 
    kernel_regularizer=regularizers.l2(best_parameters["weight_decay"])
)(x)

vision_network = keras.Model(inputs, outputs)

tf.keras.backend.clear_session()
vision_network.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_parameters["learning_rate"]),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = vision_network.fit(
    train_set,
    validation_data=validation_set,
    epochs=100,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

accuracy = history.history["accuracy"]
validation_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
validation_loss = history.history["val_loss"]
epochs_range = range(len(accuracy))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label="Training Accuracy")
plt.plot(epochs_range, validation_accuracy, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, validation_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()


vision_network.save("final_model.keras")

# converting to TFLite for loading into mobile app
converter = tf.lite.TFLiteConverter.from_keras_model(vision_network)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("final_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as final_model.tflite")


# Evaluating on test set

In [ ]:
test_loss, test_accuracy = vision_network.evaluate(test_set)
print(f"Test accuracy: {test_accuracy:.4f}")